In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import json

from DataAnalysisClass import *
from clustering_work import *
from pathlib import Path
from matplotlib import pyplot as plt
import seaborn as sns

single = ['aug-cc-pVDZ', 'aug-cc-pVTZ', 'aug-cc-pVQZ', 'aug-cc-pV5Z', 'aug-cc-pV6Z']
single_polarized = ['aug-cc-pCVDZ', 'aug-cc-pCVTZ', 'aug-cc-pCVQZ']
double = ['d-aug-cc-pVDZ', 'd-aug-cc-pVTZ', 'd-aug-cc-pVQZ', 'd-aug-cc-pV5Z', 'd-aug-cc-pV6Z']
double_polarized = ['d-aug-cc-pCVDZ', 'd-aug-cc-pCVTZ', 'd-aug-cc-pCVQZ']
all_basis_sets = single + single_polarized + double + double_polarized

In [2]:

data_base=Path("/home/ahurta92/data/may2024")
mol_list_path = data_base.joinpath("mols_list")
# read mol_list

with open(mol_list_path) as mol_list:
    full_mol_list = mol_list.read().splitlines()

# split into N lists

def split_list(lst, n):
    k, m = divmod(len(lst), n)
    return (lst[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))


In [3]:
N=5
mol_list = list(split_list(full_mol_list, N))


for i in range(N):
    print(' '.join(mol_list[i]))

FNO CH3SH N2H2 CH3NH2 HOCl SiO HBS HBO CH2BH PH3O ClF HF BH2Cl SiH4 SF2
P2H4 NaCN OCl2 CH3Cl SCl2 HCCF LiH BF SiH3F HCONH2 CS S2H2 FCN NaCl SiH3Cl
N2H4 NH2Cl LiCl NH3O NH2OH BH3 SO2 HCN CH3BH2 NaLi HOF CH3F HNS CH3OH
O3 CH2NH ClCN Mg2 NH3 H2O PH3 BHF2 LiCN NH2F HCl HCCCl HCOOH HCHS
LiH_s HCHO SH2 CSO HOOH CO HCP NOCl HNO NaH OF2 HNC BH2F LiBH4


In [4]:
from quantumresponsepro.dalton.daltonToJson import daltonToJson
from pathlib import Path
dalton_data_root=Path("/home/ahurta92/data/beta_paper_database")






In [42]:
mol='CO'
basis='aug-cc-pVDZ'
test_dalton_path=Path('/home/ahurta92/data/beta_paper_database/dalton/hf/{}/dipole/quad_{}-{}.out'.format(mol,mol,basis))
output_json_path=Path('/home/ahurta92/data/beta_paper_database/dalton/hf/{}/dipole/quad_{}-{}.json'.format(mol,mol,basis))
test_dalton_path
dtj=daltonToJson()

output_file=test_dalton_path

def read_quad_basis_data(output_file,output_json):

    with open(output_file, "r") as daltonOutput:
        dj = daltonToJson()
        dalton_json = json.loads(dj.convert(daltonOutput))
        # get the quad data df
        quad_data = dj.readQuadResponse(output_file)
        alpha_data = dj.readAlphaFromQUADResponse(output_file)
        quad_data['molecule'] = mol
        quad_data['basis'] = basis
    
        alpha_data['molecule'] = mol
        alpha_data['basis'] = basis
    
        dalton_json['alpha'] = alpha_data.to_dict()
        dalton_json['Quad'] = quad_data.to_dict()
         # write the dalton_json to file
        with(open(output_json, "w")) as f:
            json.dump(dalton_json, f, indent=4)


read_quad_basis_data(test_dalton_path,output_json_path)

test_json=json.loads(output_json_path.read_text())

READING QUAD RESPONSE from /home/ahurta92/data/beta_paper_database/dalton/hf/CO/dipole/quad_CO-aug-cc-pVDZ.out
  Results from quadratic response calculation



In [43]:
test_json

{'simulation': {'calculations': [{'calculationType': 'Dipole',
    'calculationResults': {'x': '0.00000000',
     'y': '0.00000000',
     'z': '17.05591208'}},
   {'calculationType': 'energyCalculation',
    'calculationTime': {'cpuTime': 0.154,
     'wallTime': 0.157,
     'units': 'second'},
    'calculationResults': {'totalEnergy': {'value': -112.75470171481,
      'units': 'Hartree'},
     'nuclearRepulsionEnergy': {'value': 22.514187355866, 'units': 'Hartree'},
     'electronEnergy': {'value': -135.268889070676, 'units': 'Hartree'}},
    'calculationSetup': {'numberOfElectrons': 14,
     'molecularSpinMultiplicity': 1,
     'waveFunctionType': 'HF',
     'waveFunctionTheory': 'Hartree-Fock',
     'charge': 0}}],
  'simulationTime': {'cpuTime': 3.34, 'wallTime': 3.34, 'units': 'second'}},
 'alpha': {'ij': {'0': 'XX',
   '1': 'YX',
   '2': 'ZX',
   '3': 'XX',
   '4': 'YX',
   '5': 'ZX',
   '6': 'XX',
   '7': 'YX',
   '8': 'ZX',
   '9': 'XX',
   '10': 'YX',
   '11': 'ZX',
   '12': 'X

In [44]:
def reconstruct_full_beta_ijk(freq_data):
    ex = freq_data.copy()
    ex['equal_ijk'] = freq_data['Beta'].apply(lambda x: str(x) if not isinstance(x, float) else str(None))
    ex['equal_ijk'] = ex['equal_ijk'].apply(lambda x: (''.join(x[5:-1].split(','))) if str(None) else str(None))
    ex.set_index('ijk', inplace=True)
    for index, row in ex.iterrows():
        if row.equal_ijk != str():
            ex.at[index, 'Beta'] = ex.at[row.equal_ijk, 'Beta']
    ex.reset_index(inplace=True)
    ex.drop(columns='equal_ijk', inplace=True)
    return ex


def read_basis_quad(json_file):
    basis_data = pd.DataFrame.from_dict(json_file['Quad'])
    basis_data.rename(columns={'A-freq': 'Afreq', 'B-freq': 'Bfreq', 'C-freq': 'Cfreq'}, inplace=True)
    # round Afreq Bfreq Cfreq to 3 decimal places
    basis_data.Afreq = basis_data.Afreq.round(3)
    basis_data.Bfreq = basis_data.Bfreq.round(3)
    basis_data.Cfreq = basis_data.Cfreq.round(3)
    # combine A-freq B-freq C-freq to ijk
    basis_data['ijk'] = basis_data['A'].astype(str) + basis_data['B'].astype(str) + basis_data['C'].astype(str)
    basis_data = basis_data.drop(columns=['A', 'B', 'C'])
    # rename Beta Value to Beta 
    basis_data.rename(columns={'Beta Value': 'Beta'}, inplace=True)
    bfreqs = basis_data.Bfreq.unique()
    cfreqs = basis_data.Cfreq.unique()

    ex = basis_data.copy()
    ex['equal_ijk'] = ex['Beta'].apply(lambda x: str(x) if not isinstance(x, float) else str(None))
    ex['equal_ijk'] = ex['equal_ijk'].apply(lambda x: (''.join(x[5:-1].split(','))) if str(None) else str(None))
    ex.set_index(['ijk','Bfreq','Cfreq'], inplace=True)
    ex.sort_index(inplace=True)
    ex=ex[~ex.index.duplicated(keep='first')]
    for index, row in ex.iterrows():
        if row.equal_ijk != str():
            # create new index = to equal_ijk,Cfreq,Bfreq
            new_index= (row.equal_ijk, index[2], index[1])
            ex.loc[index,'Beta'] = ex.loc[new_index,'Beta']
    ex.reset_index(inplace=True)
    ex.drop(columns='equal_ijk', inplace=True)
    return ex
    

In [45]:
def query_beta_data(df,omega_b,omega_c):
    om=df.Bfreq.unique()

    b = om[omega_b]
    c = om[omega_c]

    return df.query('Bfreq==@b and Cfreq==@c')
def query_alpha_data(df,omega):
    om=df.omega.unique()
    b = om[omega]
    return df.query('omega==@b')
def process_beta_df(beta_df):
    beta_df.Afreq = beta_df.Afreq.round(3)
    beta_df.Bfreq = beta_df.Bfreq.round(3)
    beta_df.Cfreq = beta_df.Cfreq.round(3)
    beta_df['ijk'] = beta_df['A'].astype(str) + beta_df['B'].astype(str) + beta_df['C'].astype(str)
    beta_df = beta_df.drop(columns=['A', 'B', 'C'])
    return beta_df


class basis_data:
    def __init__(self,mol,basis_set,xc,op,data_base,reread=False):
        self.mol = mol
        self.xc=xc
        self.op=op
        self.basis_set=basis_set
        # data is stored in /dalton_root/xc/mol/op/
        # and the basis set data is in quad_{mol}-{basis_set}.json
        self.data_dir = data_base
        self.output_path=self.data_dir.joinpath('dalton/{}/{}/dipole/quad_{}-{}.out'.format(xc,mol,mol,basis))
        self.output_json_path=self.data_dir.joinpath('dalton/{}/{}/dipole/quad_{}-{}.json'.format(xc,mol,mol,basis))
        print(self.output_path)

        

        if not self.output_json_path.exists() or reread==True:
            read_quad_basis_data(self.output_path,self.output_json_path)
            self.output_json=json.load(open(self.output_json_path))
        else:
            self.output_json=json.load(open(self.output_json_path))
        print(self.output_json)

        try:
            self.alpha=pd.DataFrame(self.output_json['alpha'])
            self.beta=read_basis_quad(self.output_json)
            self.energy=self.output_json['simulation']['calculations'][1]['calculationResults']['totalEnergy']['value']
        except:
            print('error reading data')
            self.alpha=None
            self.beta=None
            self.energy=None

        #calc_path = self.data_dir.joinpath("calc_path.json")
        #self.calc_path_json = json.load(open(calc_path))
        #self.alpha_json=pd.DataFrame(json.load(open(self.data_dir.joinpath(self.calc_path_json['alpha_json_path']))))
        #self.beta_json=process_beta_df(pd.DataFrame(json.load(open(self.data_dir.joinpath(self.calc_path_json['quadratic_json_path'])))))
    def get_beta(self,omega_b,omega_c):
        return query_beta_data(self.beta,omega_b,omega_c)
    def get_alpha(self,omega):
        return query_alpha_data(self.alpha,omega)
mol='BH3'
basis='aug-cc-pVDZ'
xc='hf'
op='dipole'
        


In [57]:
mol='SiO'
basis='d-aug-cc-pVQZ'
xc='hf'
op='dipole'

bdata=basis_data(mol,basis,xc,op,dalton_data_root,True)
bdata.get_beta(0,0)

/home/ahurta92/data/beta_paper_database/dalton/hf/SiO/dipole/quad_SiO-d-aug-cc-pVQZ.out
READING QUAD RESPONSE from /home/ahurta92/data/beta_paper_database/dalton/hf/SiO/dipole/quad_SiO-d-aug-cc-pVQZ.out
  Results from quadratic response calculation

{'simulation': {'calculations': [{'calculationType': 'Dipole', 'calculationResults': {'x': '0.00000000', 'y': '0.00000000', 'z': '22.82335623'}}, {'calculationType': 'energyCalculation', 'calculationTime': {'cpuTime': 19.985, 'wallTime': 20.058, 'units': 'second'}, 'calculationResults': {'totalEnergy': {'value': -363.851072283965, 'units': 'Hartree'}, 'nuclearRepulsionEnergy': {'value': 39.258029835683, 'units': 'Hartree'}, 'electronEnergy': {'value': -403.109102119648, 'units': 'Hartree'}}, 'calculationSetup': {'numberOfElectrons': 22, 'molecularSpinMultiplicity': 1, 'waveFunctionType': 'HF', 'waveFunctionTheory': 'Hartree-Fock', 'charge': 0}}], 'simulationTime': {'cpuTime': 6.0, 'wallTime': 6.0, 'units': 'second'}}, 'alpha': {'ij': {'0': 

,ijk,Bfreq,Cfreq,Afreq,Beta,molecule,basis
0,XXX,0.0,0.0,-0.0,0.0,SiO,d-aug-cc-pVQZ
25,XXY,0.0,0.0,-0.0,-0.0,SiO,d-aug-cc-pVQZ
50,XXZ,0.0,0.0,-0.0,22.024931,SiO,d-aug-cc-pVQZ
75,XYX,0.0,0.0,-0.0,-0.0,SiO,d-aug-cc-pVQZ
100,XYY,0.0,0.0,-0.0,0.0,SiO,d-aug-cc-pVQZ
125,XYZ,0.0,0.0,-0.0,-0.002986,SiO,d-aug-cc-pVQZ
150,XZX,0.0,0.0,-0.0,22.024931,SiO,d-aug-cc-pVQZ
175,XZY,0.0,0.0,-0.0,-0.002986,SiO,d-aug-cc-pVQZ
200,XZZ,0.0,0.0,-0.0,0.0,SiO,d-aug-cc-pVQZ
225,YXX,0.0,0.0,-0.0,-0.0,SiO,d-aug-cc-pVQZ


,ijk,Bfreq,Cfreq,Afreq,Beta,molecule,basis
24,XXX,0.037,0.037,-0.074,-6.087941,FNO,d-aug-cc-pVQZ
49,XXY,0.037,0.037,-0.074,-10.573077,FNO,d-aug-cc-pVQZ
74,XXZ,0.037,0.037,-0.074,-0.0,FNO,d-aug-cc-pVQZ
99,XYX,0.037,0.037,-0.074,-10.573077,FNO,d-aug-cc-pVQZ
124,XYY,0.037,0.037,-0.074,-2.80322,FNO,d-aug-cc-pVQZ
149,XYZ,0.037,0.037,-0.074,-0.0,FNO,d-aug-cc-pVQZ
174,XZX,0.037,0.037,-0.074,-0.0,FNO,d-aug-cc-pVQZ
199,XZY,0.037,0.037,-0.074,-0.0,FNO,d-aug-cc-pVQZ
224,XZZ,0.037,0.037,-0.074,2.395918,FNO,d-aug-cc-pVQZ
249,YXX,0.037,0.037,-0.074,-10.655078,FNO,d-aug-cc-pVQZ


In [11]:
data={}
output_data=data_base.joinpath("output")
available_data = []
not_available_data = []

for mol in full_mol_list:
    try:
        data[mol]=basis_data(mol,basis,xc,op,dalton_data_root,True)
        available_data.append(mol)
    except FileNotFoundError as e:
        not_available_data.append(mol)
    except:
        data[mol]=None

READING QUAD RESPONSE from /home/ahurta92/data/beta_paper_database/dalton/hf/FNO/dipole/quad_FNO-aug-cc-pVDZ.out
  Results from quadratic response calculation

READING QUAD RESPONSE from /home/ahurta92/data/beta_paper_database/dalton/hf/CH3SH/dipole/quad_CH3SH-aug-cc-pVDZ.out
  Results from quadratic response calculation

error reading data
READING QUAD RESPONSE from /home/ahurta92/data/beta_paper_database/dalton/hf/N2H2/dipole/quad_N2H2-aug-cc-pVDZ.out
  Results from quadratic response calculation

error reading data
READING QUAD RESPONSE from /home/ahurta92/data/beta_paper_database/dalton/hf/CH3NH2/dipole/quad_CH3NH2-aug-cc-pVDZ.out
  Results from quadratic response calculation

error reading data
READING QUAD RESPONSE from /home/ahurta92/data/beta_paper_database/dalton/hf/HOCl/dipole/quad_HOCl-aug-cc-pVDZ.out
  Results from quadratic response calculation

error reading data
READING QUAD RESPONSE from /home/ahurta92/data/beta_paper_database/dalton/hf/SiO/dipole/quad_SiO-aug-cc-pVDZ.o

In [25]:
data['FNO'].energy

-228.65430682395

In [63]:
full_mol_list

['FNO',
 'CH3SH',
 'N2H2',
 'CH3NH2',
 'HOCl',
 'SiO',
 'HBS',
 'HBO',
 'CH2BH',
 'PH3O',
 'ClF',
 'HF',
 'BH2Cl',
 'SiH4',
 'SF2',
 'P2H4',
 'NaCN',
 'OCl2',
 'CH3Cl',
 'SCl2',
 'HCCF',
 'LiH',
 'BF',
 'SiH3F',
 'HCONH2',
 'CS',
 'S2H2',
 'FCN',
 'NaCl',
 'SiH3Cl',
 'N2H4',
 'NH2Cl',
 'LiCl',
 'NH3O',
 'NH2OH',
 'BH3',
 'SO2',
 'HCN',
 'CH3BH2',
 'NaLi',
 'HOF',
 'CH3F',
 'HNS',
 'CH3OH',
 'O3',
 'CH2NH',
 'ClCN',
 'Mg2',
 'NH3',
 'H2O',
 'PH3',
 'BHF2',
 'LiCN',
 'NH2F',
 'HCl',
 'HCCCl',
 'HCOOH',
 'HCHS',
 'LiH_s',
 'HCHO',
 'SH2',
 'CSO',
 'HOOH',
 'CO',
 'HCP',
 'NOCl',
 'HNO',
 'NaH',
 'OF2',
 'HNC',
 'BH2F',
 'LiBH4']

In [26]:
data['FNO'].get_beta(4,4)

,ijk,Bfreq,Cfreq,Afreq,Beta,molecule,basis
24,XXX,0.037,0.037,-0.074,-9.771601,FNO,aug-cc-pVDZ
49,XXY,0.037,0.037,-0.074,-9.423197,FNO,aug-cc-pVDZ
74,XXZ,0.037,0.037,-0.074,0.0,FNO,aug-cc-pVDZ
99,XYX,0.037,0.037,-0.074,-9.423197,FNO,aug-cc-pVDZ
124,XYY,0.037,0.037,-0.074,-0.0986,FNO,aug-cc-pVDZ
149,XYZ,0.037,0.037,-0.074,-0.0,FNO,aug-cc-pVDZ
174,XZX,0.037,0.037,-0.074,0.0,FNO,aug-cc-pVDZ
199,XZY,0.037,0.037,-0.074,-0.0,FNO,aug-cc-pVDZ
224,XZZ,0.037,0.037,-0.074,2.106403,FNO,aug-cc-pVDZ
249,YXX,0.037,0.037,-0.074,-9.586072,FNO,aug-cc-pVDZ
